In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from googlenewsdecoder import gnewsdecoder
from urllib.parse import urlparse
from fake_useragent import UserAgent

In [9]:
import feedparser
from newspaper import Article
from urllib.parse import unquote

In [10]:
#Getting top news from google news (many links to sites)

def get_news(rss_url="https://news.google.com/rss?hl=en-US&gl=US&ceid=US:en"):
    response = requests.get(rss_url)
    soup = BeautifulSoup(response.content, 'xml')
    
    return [{
        'title': item.title.text,
        'source': item.source.text,
        'time': item.pubDate.text,
        'link': item.link.text
    } for item in soup.find_all('item')]

In [11]:
#Decoding the google news links to original source

def decode_google_url(google_url):
    """Decode Google News tracking URL to original source"""
    try:
        # Extract article ID from URL path
        path_segments = urlparse(google_url).path.split('/')
        article_id = path_segments[-1] if path_segments else ""
        
        # Use the decoder package
        result = gnewsdecoder(
            google_url,
            interval=2,  # Add 2s delay between requests
            proxy=None  # Add proxy if getting blocked
        )
        
        if result['status']:
            return result['decoded_url']
        return google_url  # Fallback to original URL
    
    except Exception as e:
        print(f"Decoding failed: {str(e)}")
        return google_url


In [12]:
news=get_news()
news

[{'title': "Trump says judge blocking deportations 'should be impeached,' DOJ challenges jurisdiction - CNBC",
  'source': 'CNBC',
  'time': 'Tue, 18 Mar 2025 14:28:53 GMT',
  'link': 'https://news.google.com/rss/articles/CBMikwFBVV95cUxNbUNTMGkzVm9KeTZLVGhHVzhfUzRJODdtRmdTX1hBVUdWeFN3NFJUWmgxT2JXdW1teldjd3dMY2JJM3A5QmZQd2UxRk9JSFRfZnBFYk84Zk5FSjRmaTNSaDNMeWp4c21NX2ZCWmNEbnlYX2syRjVicVppbG45WXdsOVRQMGF6REQxYjJhTVpNR0Vzb03SAZgBQVVfeXFMT1JuQkVGT01jN1pZYTIyYi12RWtXRDk2MDY2LWhZaC1DUjVGU2Fza2pmZFhZdzBsakJnamVGUDQ4TnhodGdSSXp5ZFlwZWdEb1dsbnBuMHdrMEh5QVBGVmNidTY3azNXRnpGUEs1SENvaFJYcHhJZElDdWo4Q1NzdENOVXV1b0lYdGxPVXFWWUZST0UwTDBiQjQ?oc=5'},
 {'title': 'Trump and Putin discuss Ukraine war, White House says the talks are going well - Reuters',
  'source': 'Reuters',
  'time': 'Tue, 18 Mar 2025 15:39:40 GMT',
  'link': 'https://news.google.com/rss/articles/CBMiowFBVV95cUxOdGtkRTVMMTUxS1FvT3NqWXdFMG9DRFV0XzJFR0JRUWFNaGJfMF9MUWFub2dxcVIyTEVFTHhLTDVUa180MVVPOWxQUTYyQkMtVjVBajM3UVZSWXM5elE3S1BsbzYyX

In [13]:
for item in news:
    item['link'] = decode_google_url(item['link'])
    print(item['link'])

https://www.cnbc.com/2025/03/18/trump-impeachment-obama-judge-deportations-venezuela.html
https://www.reuters.com/world/europe/trump-hold-call-with-putin-test-deal-making-strength-2025-03-18/
https://www.bbc.com/news/articles/cq6yp5d5v9jo
https://www.reuters.com/legal/trump-administration-reinstating-24500-fired-workers-after-court-order-2025-03-18/
https://www.seattletimes.com/opinion/science-and-reason-overboard/
https://www.foxnews.com/politics/checks-balances-trump-supporters-seek-push-back-against-activist-judges
https://abcnews.go.com/International/parents-missing-pittsburgh-student-dominican-police-declare-dead/story?id=119904931
https://www.nytimes.com/2025/03/18/us/politics/jfk-assassination-documents-trump.html
https://www.washingtonpost.com/entertainment/theater/2025/03/17/trump-kennedy-center-honors-board-meeting/
https://www.nytimes.com/2025/03/17/us/rasha-alawieh-brown-university-doctor-deported-hezbollah.html
https://www.ft.com/content/80742c32-1af3-4881-a935-f3045df12b1

In [14]:
news

[{'title': "Trump says judge blocking deportations 'should be impeached,' DOJ challenges jurisdiction - CNBC",
  'source': 'CNBC',
  'time': 'Tue, 18 Mar 2025 14:28:53 GMT',
  'link': 'https://www.cnbc.com/2025/03/18/trump-impeachment-obama-judge-deportations-venezuela.html'},
 {'title': 'Trump and Putin discuss Ukraine war, White House says the talks are going well - Reuters',
  'source': 'Reuters',
  'time': 'Tue, 18 Mar 2025 15:39:40 GMT',
  'link': 'https://www.reuters.com/world/europe/trump-hold-call-with-putin-test-deal-making-strength-2025-03-18/'},
 {'title': 'Gaza: Why the Hamas-Israel ceasefire has not held - BBC.com',
  'source': 'BBC.com',
  'time': 'Tue, 18 Mar 2025 14:48:41 GMT',
  'link': 'https://www.bbc.com/news/articles/cq6yp5d5v9jo'},
 {'title': 'Trump administration reinstating almost 25,000 fired workers after court order - Reuters',
  'source': 'Reuters',
  'time': 'Tue, 18 Mar 2025 16:05:20 GMT',
  'link': 'https://www.reuters.com/legal/trump-administration-reins

In [15]:
import requests
from newspaper import Article
from fake_useragent import UserAgent
from requests.adapters import HTTPAdapter
from urllib3.util import Retry
import time

# Configure session with retry logic
session = requests.Session()
retries = Retry(
    total=3,
    backoff_factor=0.5,
    status_forcelist=[500, 502, 503, 504],
    allowed_methods=["GET"]
)
session.mount('https://', HTTPAdapter(max_retries=retries))

def get_full_article_content(article_url):
    """Enhanced article downloader with timeout handling"""
    headers = {
        'User-Agent': UserAgent().random,
        'Accept-Language': 'en-US,en;q=0.9',
        'Referer': 'https://news.google.com/',
        'DNT': '1'
    }
    
    try:
        # Get article with timeout and retry logic
        response = session.get(article_url, headers=headers, timeout=(3.05, 10))
        response.raise_for_status()

        article = Article(article_url)
        article.download(input_html=response.text)
        article.parse()
        
        return {
            'title': article.title,
            'publish_date': article.publish_date,
            'text': article.text,
            'top_image': article.top_image,
            'source_url': article_url,
        }

    except requests.exceptions.Timeout:
        print(f"⏰ Timeout skipped: {article_url}")
        return None
    except Exception as e:
        print(f"⚠️ Error processing {article_url}: {str(e)}")
        return None

# Batch processing with progress tracking
def process_news_items(news_items, delay=1):
    results = []
    for idx, item in enumerate(news_items, 1):
        print(f"Processing article {idx}/{len(news_items)}...")
        article = get_full_article_content(item['link'])
        if article:
            results.append(article)
        time.sleep(delay)  # Respectful delay between requests
    return results

# Usage
newslist = process_news_items(news)
newslist

Processing article 1/38...
Processing article 2/38...
Processing article 3/38...
Processing article 4/38...
Processing article 5/38...
Processing article 6/38...
Processing article 7/38...
Processing article 8/38...
Processing article 9/38...
Processing article 10/38...
Processing article 11/38...
Processing article 12/38...
Processing article 13/38...
Processing article 14/38...
Processing article 15/38...
Processing article 16/38...
Processing article 17/38...
Processing article 18/38...
Processing article 19/38...
Processing article 20/38...
Processing article 21/38...
Processing article 22/38...
Processing article 23/38...
Processing article 24/38...
Processing article 25/38...
Processing article 26/38...
Processing article 27/38...
Processing article 28/38...
Processing article 29/38...
Processing article 30/38...
Processing article 31/38...
Processing article 32/38...
Processing article 33/38...
Processing article 34/38...
Processing article 35/38...
Processing article 36/38...
P

[{'title': 'Supreme Court chief justice rebukes Trump over call to impeach judge in deportation case',
  'publish_date': datetime.datetime(2025, 3, 18, 0, 0),
  'text': 'John Roberts, chief justice of the U.S. Supreme Court, during the 60th presidential inauguration on Jan. 20, 2025.\n\nSupreme Court Chief Justice John Roberts rebuked President Donald Trump over calling for the impeachment of a federal judge who has issued orders blocking Trump\'s plan to deport any more alleged members of a Venezuelan gang from the United States.\n\n"For more than two centuries, it has been established that impeachment is not an appropriate response to disagreement concerning a judicial decision," Roberts said in a statement on Trump\'s demand.\n\n"The normal appellate review process exists for that purpose," Roberts said, several hours after Trump said that Chief Judge James Boasberg should be impeached.\n\nTrump in a Truth Social post had written, "This judge, like many of the Crooked Judges\' I am 

In [16]:
df = pd.DataFrame(newslist)

In [17]:
df

,title,publish_date,text,top_image,source_url
0,Supreme Court chief justice rebukes Trump over...,2025-03-18 00:00:00,"John Roberts, chief justice of the U.S. Suprem...",https://image.cnbcfm.com/api/v1/image/10810819...,https://www.cnbc.com/2025/03/18/trump-impeachm...
1,Trump and Putin conclude call on moves to end ...,2025-03-18 00:00:00,U.S. President Donald Trump waves as he return...,https://www.reuters.com/resizer/v2/IRYT7BOZAJI...,https://www.reuters.com/world/europe/trump-hol...
2,Gaza ceasefire deal: Why the Hamas-Israel truc...,None,Why has Israel bombed Gaza and what next for c...,https://ichef.bbci.co.uk/news/1024/branded_new...,https://www.bbc.com/news/articles/cq6yp5d5v9jo
3,"Trump administration reinstating almost 25,000...",2025-03-18 00:00:00,Members of the U.S. Capitol Police standby as ...,https://www.reuters.com/resizer/v2/EAU6LIV46FL...,https://www.reuters.com/legal/trump-administra...
4,Science and reason overboard!,2025-03-13 15:04:00-07:00,IN THE NEWS: The Trump administration’s crusad...,https://images.seattletimes.com/wp-content/upl...,https://www.seattletimes.com/opinion/science-a...
5,"Checks and balances: Trump, supporters seek to...",None,As the lawsuits filed against President Donald...,https://static.foxnews.com/foxnews.com/content...,https://www.foxnews.com/politics/checks-balanc...
6,Parents of missing Pittsburgh student ask Domi...,None,"Sudiksha Konanki, 20, disappeared in Punta Can...",https://i.abcnewsfe.com/a/7715a296-18cf-4ce2-a...,https://abcnews.go.com/International/parents-m...
7,"Trump Promises, Again, to Release ‘All’ Kenned...",2025-03-18 00:00:00,President Trump said officials planned to disc...,https://static01.nyt.com/images/2025/03/18/mul...,https://www.nytimes.com/2025/03/18/us/politics...
8,"At Kennedy Center, Trump envisions remaking Ho...",2025-03-17 00:00:00,President Donald Trump visited the Kennedy Cen...,https://www.washingtonpost.com/wp-apps/imrs.ph...,https://www.washingtonpost.com/entertainment/t...
9,Deported Professor Rasha Alawieh Attended Hezb...,2025-03-17 00:00:00,The Department of Homeland Security said on Mo...,https://static01.nyt.com/images/2025/03/17/mul...,https://www.nytimes.com/2025/03/17/us/rasha-al...
